## Importing xlsx

In [1]:
#%pip install openpyxl
#%pip install xlrd

In [2]:
import pandas as pd
import numpy as np
from apifunctions import *
import datetime

pd.set_option("display.max_rows", None, "display.max_columns", None)



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/101.0.4951.41/chromedriver_mac64.zip
Driver has been saved in cache [/Users/danigomezlechonbarrachina/.wdm/drivers/chromedriver/mac64/101.0.4951.41]


In [3]:
wacc=pd.read_excel("../data/datos_Bloomberg/wacc.xlsx")
prices=pd.read_excel("../data/datos_Bloomberg/prices.xlsx")
sales_growth=pd.read_excel("../data/datos_Bloomberg/sales_growth.xlsx")
parameters=pd.read_excel("../data/datos_Bloomberg/parameters.xlsx")

In [4]:
company_list=wacc.ID

## ID's as index

In [5]:
datasets=[wacc,prices,sales_growth]

for i in datasets:
    i.set_index("ID",inplace=True)

## Dealing with NANs

In [6]:
# We first want to delete the rows which have all NANs for the wacc, prices and sales_growth dataframes

datasets=[wacc,prices,sales_growth]

for i in datasets:
    i.dropna(axis=0, how='all',inplace=True)   

#### For simplicity, in prices we are going to delete all the rows that have nans as that means the company was not trading at that point so it wont allow us to do the backtesting:

In [7]:
prices.dropna(axis=0,inplace=True)

#### Now we are going to delete the companies for which we dont have information in one of the three datasets

In [8]:
remove_columns(prices,wacc,sales_growth)
remove_columns(prices,wacc,sales_growth)

#### To deal with the rest of NANs in wacc and sales_growth we are going to fill them with the average of the values in its time range

In [9]:
fill_na_mean(wacc)
fill_na_mean(sales_growth)

#### At this point we have cleaned the three dataframes: wacc, prices, sales growth. Now we need to deal with the parameters one

In [10]:
final_company_list=wacc.index

In [11]:
deleted_companies=[]

for i in company_list:
    
    if i not in final_company_list:
        
        deleted_companies.append(i)

### Parameters dataframe

#### We first create column with the year only and we set the dates as index and transpose the dataframe to have it in the same format as the rest:

In [12]:
lista=[]

for i in parameters["Unnamed: 0"]:
    
    lista.append(str(i))

year=[]
for j in lista:
    year.append(j[:4])
    
    

In [13]:
parameters["dates"]=year
parameters_new=parameters.groupby("dates").sum()

In [14]:
parameters_new_t=parameters_new.transpose()

In [15]:
parameters_new_t["company_name"]=parameters_new_t.index

#### we add a column named company name with the ticker of each company

In [16]:
parameters_new_t["company_name"]=parameters_new_t["company_name"].str.replace('\.\d+', '')

#### As we dont have values for 2022 we drop the entire column

In [17]:
parameters_new_t=parameters_new_t.drop(["2022"],axis=1)

#### We convert the numbers to float and then delete the rows which have at least one missing value

In [18]:
#parameters_new_t[["2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","2021"]].astype(dtype="float64")

In [19]:
parameters_new_t=parameters_new_t[~(parameters_new_t[["2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","2021"]]==0).any(axis=1)]

#### Now we need to check for which companies we have full data

In [20]:
companies_we_have=parameters_new_t["company_name"]

In [21]:
final_comps=["IFF UN Equity ","NTAP UW Equity","ATO UN Equity","GWW UN Equity",     "UHS UN Equity ",     "WEC UN Equity ",     "SWKS UW Equity",     "HSY UN Equity ",     "DGX UN Equity",     "HRL UN Equity",     "EW UN Equity" ,      "MDLZ UW Equity" ,    "ATVI UW Equity",     "HUM UN Equity",      "ROK UN Equity",      "ITW UN Equity",     "TT UN Equity",       "IPG UN Equity",      "WM UN Equity",       "J UN Equity",        "BR UN Equity",       "GPC UN Equity",      "GIS UN Equity",      "GD UN Equity",       "KLAC UW Equity",     "COST UW Equity",     "PCAR UW Equity",     "GOOG UW Equity",     "AEE UN Equity",      "ECL UN Equity",      "EMR UN Equity",      "ODFL UW Equity",     "AON UN Equity",      "DVA UN Equity",      "CBRE UN Equity",     "MNST UW Equity",    "CTAS UW Equity",     "EL UN Equity",       "BRO UN Equity",      "AAPL UW Equity",     "AMGN UW Equity",     "KMX UN Equity",      "MA UN Equity",       "KMB UN Equity",     "PM UN Equity"  ,     "NDAQ UW Equity" ,    "SO UN Equity",   "BKNG UW Equity",  "MCO UN Equity", "OMC UN Equity","OKE UN Equity",      "PWR UN Equity",      "PH UN Equity",       "ROL UN Equity",      "PPL UN Equity",      "AME UN Equity",      "NOC UN Equity",     "RSG UN Equity" ,     "ISRG UW Equity",     "PPG UN Equity",      "CTSH UW Equity",     "HSIC UW Equity",     "RHI UN Equity",      "WST UN Equity",      "FDS UN Equity",      "FFIV UW Equity",     "ES UN Equity",       "UNH UN Equity",      "CHRW UW Equity",     "LLY UN Equity",      "APH UN Equity",      "ULTA UW Equity",     "ANTM UN Equity" ,    "LOW UN Equity",      "IEX UN Equity",      "MMC UN Equity",      "SPGI UN Equity",     "ACN UN Equity",      "AKAM UW Equity",     "CVS UN Equity",      "TXN UW Equity",      "SYY UN Equity",      "GOOGL UW Equity",    "TDG UN Equity",      "YUM UN Equity",      "NKE UN Equity",      "MKTX UW Equity",     "DOV UN Equity",      "ANSS UW Equity",     "DE UN Equity",       "PAYX UW Equity",     "ORLY UW Equity",     "BWA UN Equity",      "BIIB UW Equity",     "HON UW Equity ",    "WMT UN Equity",      "CSCO UW Equity",     "INTC UW Equity",     "MSFT UW Equity ",    "DG UN Equity",       "CI UN Equity",       "HAS UW Equity",      "AMCR UN Equity",    "UPS UN Equity",      "HCA UN Equity",      "TGT UN Equity",      "MHK UN Equity" ,     "FISV UW Equity",    "PG UN Equity",       "CNC UN Equity",      "APD UN Equity",      "AWK UN Equity",      "EQIX UW Equity",     "AOS UN Equity" ,     "CAT UN Equity",      "LKQ UW Equity",     "KO UN Equity",       "APTV UN Equity" ,    "INTU UW Equity",     "FLT UN Equity",      "IDXX UW Equity",     "ROST UW Equity",     "ROP UN Equity",      "HD UN Equity",       "MPWR UW Equity",    "QCOM UW Equity",     "MCD UN Equity",      "MRK UN Equity",      "MMM UN Equity",      "STE UN Equity" ,     "WBA UW Equity",      "CPRT UW Equity",     "FAST UW Equity",     "NDSN UW Equity",     "BBY UN Equity",     "NVR UN Equity",      "ALB UN Equity" ,     "EPAM UN Equity",     "BF/B UN Equity",     "CPB UN Equity",      "IPGP UW Equity",     "MTD UN Equity",      "RMD UN Equity",      
"TSCO UW Equity",     "AMAT UW Equity",     "CLX UN Equity",      "CMS UN Equity",      "WAB UN Equity",      "POOL UW Equity",     "CAG UN Equity",      "AZO UN Equity",      "DHR UN Equity",      "CMI UN Equity",      "EXPD UW Equity",     "ADM UN Equity",      "ADP UW Equity",      "VRSK UW Equity",     "CL UN Equity",       "LMT UN Equity",      "V UN Equity",        "ED UN Equity",       "MSCI UN Equity",     
"WAT UN Equity"]      

In [22]:
companies_to_use=[]

for i in final_comps:
    
    if i in final_company_list:
        
        companies_to_use.append(i)

In [23]:
wacc=wacc/100

In [24]:
sales_growth=sales_growth/100

In [25]:
parameters_new_t[["2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","2021"]][parameters_new_t["DATE"]=="EBITDA_TO_REVENUE"]=parameters_new_t[["2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","2021"]][parameters_new_t["company_name"]=="EBITDA_TO_REVENUE"]/100

In [26]:
parameters_new_t[["2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","2021"]][parameters_new_t["DATE"]=="EBITDA_TO_REVENUE"]=parameters_new_t[["2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","2021"]]/100

In [27]:
%store wacc
%store prices
%store sales_growth
%store parameters_new_t
%store companies_to_use

Stored 'wacc' (DataFrame)
Stored 'prices' (DataFrame)
Stored 'sales_growth' (DataFrame)
Stored 'parameters_new_t' (DataFrame)
Stored 'companies_to_use' (list)


In [30]:
parameters_new_t[parameters_new_t["company_name"]=="ATO UN Equity"]

dates,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,DATE,company_name
ATO UN Equity,4719835000,4286435000,3438483000,3886257000,4940916000,4142136000,2454648000,2759735000,3115546000,2901848000,2821137000,3407490000,SALES_REV_TURN,ATO UN Equity
ATO UN Equity.1,14.706256,15.377371,20.134839,19.01076,17.513676,21.877384,38.715368,37.944911,34.954291,39.199641,45.896885,41.829147,EBITDA_TO_REVENUE,ATO UN Equity
ATO UN Equity.2,38.545725,36.037948,33.821818,38.199878,39.218655,38.313119,36.268499,36.647111,1.322111,21.359538,19.463548,18.764334,EFF_TAX_RATE,ATO UN Equity
ATO UN Equity.3,216960000,233155000,246093000,236928000,253987000,274796000,293096000,319633000,361083000,391456000,429828000,477977000,CF_DEPR_AMORT,ATO UN Equity
ATO UN Equity.4,-382406000,434242000,-591347000,152772000,160410000,-393707000,-578078000,632798000,-962481000,684869000,440266000,-360497000,CHNG_WORK_CAP,ATO UN Equity
ATO UN Equity.5,11.496927,14.533406,21.313411,21.744136,16.90478,23.263867,44.281298,41.202833,47.105419,58.35857,68.613329,57.800316,CAP_EXPEND_TO_SALES,ATO UN Equity
ATO UN Equity.6,-542636000,-622965000,-732858000,-845033000,-835251000,-963621000,-1086950000,-1137089000,-1467591000,-1693477000,-1935676000,-1969540000,NET_CAP_EXP,ATO UN Equity


In [29]:
companies_to_use

['ATO UN Equity',
 'GWW UN Equity',
 'SWKS UW Equity',
 'DGX UN Equity',
 'EW UN Equity',
 'ATVI UW Equity',
 'HUM UN Equity',
 'ROK UN Equity',
 'ITW UN Equity',
 'TT UN Equity',
 'IPG UN Equity',
 'WM UN Equity',
 'J UN Equity',
 'BR UN Equity',
 'GPC UN Equity',
 'GD UN Equity',
 'KLAC UW Equity',
 'PCAR UW Equity',
 'AEE UN Equity',
 'ECL UN Equity',
 'EMR UN Equity',
 'ODFL UW Equity',
 'AON UN Equity',
 'DVA UN Equity',
 'CBRE UN Equity',
 'MNST UW Equity',
 'EL UN Equity',
 'BRO UN Equity',
 'AAPL UW Equity',
 'AMGN UW Equity',
 'MA UN Equity',
 'KMB UN Equity',
 'PM UN Equity',
 'NDAQ UW Equity',
 'SO UN Equity',
 'BKNG UW Equity',
 'MCO UN Equity',
 'OMC UN Equity',
 'OKE UN Equity',
 'PWR UN Equity',
 'PH UN Equity',
 'ROL UN Equity',
 'PPL UN Equity',
 'AME UN Equity',
 'NOC UN Equity',
 'RSG UN Equity',
 'ISRG UW Equity',
 'PPG UN Equity',
 'CTSH UW Equity',
 'HSIC UW Equity',
 'RHI UN Equity',
 'WST UN Equity',
 'FFIV UW Equity',
 'ES UN Equity',
 'UNH UN Equity',
 'CHRW U